# Tone Spiral

Brett Deaton - Fall 2023

A representation of pure tones in shapes and sounds.

## Usage

In [22]:
scale = Major("Ab") # A flat
play(scale)
#draw(scale)

## Definitions

In [18]:
class Scale:
    def freq_from_name(self, key_name):
        pass

In [19]:
class Major(Scale):
    def __init__(self, key_name):
        self.key_name = key_name
        self.principle_tone = super().freq_from_name(self.key_name)

In [20]:
def play(scale):
    pass

In [21]:
x = Major("B")